# Kак роботы планируют траекторию движения

In [1]:
# Не изменяйте эту ячейку и запустите её перед началом работы!
!pip install pygame
import math

You are using pip version 10.0.1, however version 20.3.4 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


![1](./Image/1.png)

В этом ноутбуке мы рассмотрим проблему нахождения кратчайшего пути на карте. Эта задача используется в робототехнике, компьютерных играх и сервисах такси, доставки и т.п. В данном ноутбуке карта будет представлять из себя сетку, в которой одни клетки проходимы, а другие представляют собой препятствия. Существуют разные спрособы определять соседей клетки, например, соседями могут являться только смежные клетки с данной или можно еще разрешать ходить по диагонали при некоторых ограничениях. В этом задании будем предполагать, что мы можем ходить только по смежным клеткам: вверх, вниз, влево и вправо.

Базовым алгоритмом поиска кратчайшего пути является алгоритм Дийкстры. Суть алгоритма проста — обходить вершины в порядке возрастания расстояния от начальной вершины. Для этого мы храним для каждой вершины метку $g$ — минимальное известное расстояние от начальной вершины до этой (изначально для стартовой вершины оно равно 0, а для всех остальных $\infty$).
 На каждом шаге мы рассматриваем вершину с минимальным g среди нерассмотренных (Понятно, что на первой итерации выбрана будет стартовая вершина). Нетрудно доказать, что при неотрицательных весах при рассмотрении $g$ этой вершины равно истинному значению минимального расстояния.  Далее, на текущей итерации, из рассматриваемой вершины производятся релаксации: мы пытаемся улучшить значение g для всех соседей этой вершины, то есть
 
$$neighbour.g = min(neighbour.g,\  self.g+dist(self, neighbour)).$$

![2](./Image/dijkstra.gif)

В нашем случае мы решаем задачу поиска кратчайшего пути до одной конкретной вершины, значит во время рассмотрения этой самой вершины алгоритм можно и нужно остановить. Помимо этого в нашей ситуации много упрощений, поскольку граф клетчатый, соседи - это несколько смежных клеток, а веса рёбер равны единице.

Однако алгоритм Дийкстры имеет большой минус: если мы ищем кратчайший путь от начальной вершины до конкретной, то мы рассмотрим ВСЕ вершины, расстояние до которых меньше чем до целевой, даже если они в противоположной от неё стороне. Например, если мы хотим долететь из Новосибирска до Владивостока, то зачем лететь с пересадкой в Москве, ведь она находится в противоположной стороне. Чтобы это исправить, рассмотрим $h$ — эвристическую функцию вершины. Это оценка расстояния от неё до конечной. И тогда логично брать вершину не с минимальным $g$, а минимальным значением $f = h + g$. Такой алогоритм называется A* (A-star, А-звёздочка). Вот наглядный пример его работы:

In [2]:
# Может стоит добавить две гифки - сравнение дийкстры и a*?

![3](./Image/astar.gif)

Как видите, алгоритм не рассматривает многие вершины, которые алгоритм Дийсткры бы рассмотрел. Таким образом, хорошая метрика для A* как раз поможет избежать лишних действий

Введём класс Node, описывающий вершину. У него есть следующие поля:
- i, j - координаты вершины
- g
- f = g + h
- parent - вершина, из которой мы пришли в данную. По этому полю мы сможем восстановить путь.

In [3]:
class Node:
    def __init__(self, i, j, g = math.inf, h = math.inf, parent = None):
        self.i = i
        self.j = j
        self.g = g
        self.f = self.g + h       
        self.parent = parent
    
    def __eq__(self, other):
        return (self.i == other.i) and (self.j == other.j)
    
    def __gt__(self, other):
        return self.F - EPS > other.F
    
    def __hash__(self):
        return 31 + 7 * (hash(self.i) + 7 * hash(self.j))
    
    def __str__(self):
        return 'i: {0}, j: {1}, g: {2}, F: {3}, parent: {4}'.format(self.i, self.j, self.g, self.F, self.parent)

### Реализация Open и Closed

В нашей реализации мы будем использовать два класса — Open и Closed. Open содержит нерассмотренные вершины, а Closed — уже рассмотренные. В качестве примера приведены реализации на List-е. Однако, такая реализация крайне не эффективная. Напишите свою реализацию, использующую  более эффективную структуру данных

In [4]:
# TODO согласовать Open с бекендом Глеба

Посмотрите на импелементацию Open выше и убедитесь, что вы всё понимаете. Как видите, от Open требуется выполнять следующие операции: добавлять вершину (или обновлять значение её g) и быстро доставать ноду с минимальным f-value. Подумайте, какая структура данных могут эффективно выполнять эти операции? (Подсказка: в стандартной библиотеке Python есть подходящая)

In [5]:
# TODO согласовать Closed с бекендом Глеба

Как видно, от Closed требуется выполнять операции добавления и узнавать, есть ли элемент в множестве. Уверены, что вы знакомы с нужной струкурой данных.

## Алгоритм A*

В качестве эвристической функции будем использовать Манхэтэнское расстояние — сумму расстояний по вертикали и горизонтали.

In [6]:
def ManhattanMetric(x1, y1, x2, y2):
    return abs(x1 - x2) + abs(y1 - y2)

А теперь вам предстоит реализовать используя сам алгоритм. !!!TODO написать про его парамерты

In [7]:
# TODO согласовать astar с бекендом Глеба

In [8]:
# TODO сделать тесты